# Taxonomic novelty

TODO: check how interesting the specie is -> if we have same characterized TPS from this specie or the specie group because from some new group, there could be higher chance of finding something new

In [1]:
import pandas as pd

In [2]:
import ast

In [33]:
anno_df = pd.read_csv('../annotation/all_annotation.txt', sep='\t')

C:\Users\terca\AppData\Local\Temp\ipykernel_22828\3051009688.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  anno_df = pd.read_csv('../all_annotation.txt', sep='\t')


In [4]:
anno_df = anno_df[['id', 'organism', 'tax_id', 'lineage', 'superkingdom', 'kingdom', 'phylum']]

In [15]:
anno_df.iloc[682470]['lineage']

"{'species': 'Sorangium cellulosum', 'genus': 'Sorangium', 'family': 'Polyangiaceae', 'order': 'Polyangiales', 'phylum': 'Myxococcota', 'superkingdom': 'Bacteria'}"

In [34]:
def convert_to_dict(text):
    try:
        return ast.literal_eval(text)
    except (SyntaxError, ValueError):
        return text

In [35]:
convert_to_dict('error')

'error'

In [36]:
anno_df['lineage'] = anno_df['lineage'].apply(lambda x: convert_to_dict(x))

## Characterized sequences

In [25]:
tps_db_file = '../../../01_tps_db_analysis/data/TPS-database_filtered.tsv'

In [26]:
tps_db = pd.read_csv(tps_db_file, sep='\t')

In [40]:
tps_db['Species'] = tps_db['Species'].str.strip()

In [29]:
import taxoniq

In [48]:
id_taxid_map_df = pd.read_csv('taxo_map_char.tsv', sep='\t')[['Entry', 'Organism (ID)']]

In [62]:
def get_tax_id(x):
    try:
        t = taxoniq.Taxon(scientific_name=x['Species'])
        return t.tax_id
    except:
        if x['Uniprot ID'] in id_taxid_map_df.index:
            return id_taxid_map_df.loc[x['Uniprot ID']]['Organism (ID)']
        elif x['Species'] == 'Amycolatopis arida':
            return 587909
        elif x['Species'] == 'Bubarida sp.':
            return 2772871
        elif x['Species'] == 'Dendronephtha gigantea':
            return 151771
        elif x['Species'] == 'T. versicolor':
            return 5325
        elif x['Species'] == 'P. ostreatus':
            return 5322
        elif x['Species'] == 'Au. vulgare':
            return 40419
        elif x['Species'] == 'Luffa cylindrica':
            return 3670
        else:
            return 'error'

In [63]:
tps_db['tax_id'] = tps_db.apply(lambda x: get_tax_id(x), axis=1)

In [65]:
def add_lineage(x):
    try:
        t = taxoniq.Taxon(x)
        lineage = dict()
        for line in t.ranked_lineage:
            lineage[line.rank.name] = line.scientific_name
        return lineage
    except:
        return 'error'

In [66]:
tps_db['lineage'] = tps_db['tax_id'].apply(lambda x: add_lineage(x))

In [80]:
tps_db['lineage'].apply(lambda x: list(x.keys())).drop_duplicates().tolist()

[['species',
  'genus',
  'family',
  'order',
  'class',
  'phylum',
  'kingdom',
  'superkingdom'],
 ['species', 'genus', 'family', 'order', 'class', 'phylum', 'superkingdom'],
 ['order', 'class', 'phylum', 'kingdom', 'superkingdom'],
 ['species', 'genus', 'class', 'phylum', 'kingdom', 'superkingdom'],
 ['species', 'genus', 'family', 'order', 'phylum', 'superkingdom'],
 ['species', 'genus', 'order', 'class', 'phylum', 'kingdom', 'superkingdom']]

- species
- genus
- family
- order
- class
- phylum
- kingdom
- superkingdom

In [83]:
tps_db['genus'] = tps_db['lineage'].apply(lambda x: x.get('genus', None))

In [86]:
tps_db['family'] = tps_db['lineage'].apply(lambda x: x.get('family', None))

In [87]:
tps_db['order'] = tps_db['lineage'].apply(lambda x: x.get('order', None))

In [88]:
tps_db['class'] = tps_db['lineage'].apply(lambda x: x.get('class', None))

In [89]:
tps_db['phylum'] = tps_db['lineage'].apply(lambda x: x.get('phylum', None))

In [90]:
tps_db['kingdom'] = tps_db['lineage'].apply(lambda x: x.get('kingdom', None))

In [91]:
tps_db['superkingdom'] = tps_db['lineage'].apply(lambda x: x.get('superkingdom', None))

In [93]:
tps_db['family'].nunique()

153

In [94]:
tps_db['family'].value_counts()

family
Lamiaceae             233
Pinaceae              220
Poaceae               163
Solanaceae            160
Asteraceae             82
                     ... 
Roseiflexaceae          1
Dioscoreaceae           1
Polygalaceae            1
Dipodascaceae           1
Herpetosiphonaceae      1
Name: count, Length: 153, dtype: int64

In [135]:
tps_db['species'] = tps_db['Species']

In [152]:
tps_db.to_csv('tpd_db_with_tax_info.csv')

## Check uncharacterized phyla

In [103]:
unchar_phyla = set(anno_df['lineage'].apply(lambda x: x.get('phylum', None) if type(x) == dict else None).unique()).difference(set(tps_db['phylum'].unique()))

In [104]:
len(unchar_phyla)

217

In [107]:
anno_df[anno_df.apply(lambda x: x['lineage'].get('phylum') in unchar_phyla if type(x['lineage']) == dict else False, axis=1)]

,id,organism,tax_id,lineage,superkingdom,kingdom,phylum
0,bfd_106193,soil metagenome,410658,{'species': 'soil metagenome'},Metagenome,environmental,unknown
1,bfd_118229,soil metagenome,410658,{'species': 'soil metagenome'},Metagenome,environmental,unknown
3,bfd_096880,Absidia repens,90262,"{'species': 'Absidia repens', 'genus': 'Absidi...",Eukaryota,Fungi,Mucoromycota
4,bfd_097326,soil metagenome,410658,{'species': 'soil metagenome'},Metagenome,environmental,unknown
6,bfd_203259,soil metagenome,410658,{'species': 'soil metagenome'},Metagenome,environmental,unknown
...,...,...,...,...,...,...,...
682457,uniprot_238349,Candidatus Poseidoniales archaeon,2163009,{'species': 'Candidatus Poseidoniales archaeon...,Archaea,unknown,Candidatus Thermoplasmatota
682458,uniprot_137796,Fragilariopsis cylindrus,186039,"{'species': 'Fragilariopsis cylindrus', 'genus...",Eukaryota,unknown,Bacillariophyta
682462,uniprot_132901,Helicobacter pylori,210,"{'species': 'Helicobacter pylori', 'genus': 'H...",Bacteria,unknown,Campylobacterota
682464,uniprot_208094,Luteitalea sp.,2004800,"{'species': 'Luteitalea sp.', 'genus': 'Luteit...",Bacteria,unknown,Acidobacteriota


## Scoring
- 0 for characterized specie
- 0.1 for uncharacterized species (unless it is 'unknown' or metagenomic)
- 0.2 for uncharacterized genus
- 0.4 for uncharacterized family
- 0.6 for uncharacterized order
- 0.8 for uncharacterized class
- 1 for uncharacterized phylum


In [110]:
char_phyla = tps_db['phylum'].unique()

In [116]:
char_phyla

array(['Streptophyta', 'Cnidaria', 'Actinomycetota', 'Bacteroidota',
       'Porifera', 'Ascomycota', 'Basidiomycota', 'Chlorophyta',
       'Cyanobacteriota', 'Chordata', 'Bacillota', 'Pseudomonadota',
       'Arthropoda', 'Evosea', 'Myxococcota', 'Chloroflexota',
       'Rhodophyta'], dtype=object)

In [111]:
def uncharacterized_tax_group(x, tax_group='phylum'):
    if type(x) == dict:
        value = x.get(tax_group, None)
        
        # The specie does not have the phylum information
        if value is None:
            return None
        
        # Check if there is characterized TPS from the same phylum 
        if value in tps_db[tax_group].unique():
            return False
        else:
            return True
    else:
        return None

In [119]:
anno_df['unchar_phylum'] = anno_df['lineage'].apply(lambda x: uncharacterized_tax_group(x, 'phylum'))

C:\Users\terca\AppData\Local\Temp\ipykernel_856\153500747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anno_df['unchar_phylum'] = anno_df['lineage'].apply(lambda x: uncharacterized_tax_group(x, 'phylum'))


There are 35541 sequences from phyla not present in the TPS db.
These are 216 distinct phyla. The sequences are in 60% from Bacteria, 20% Archaea and 20% Eukarya.

In [126]:
anno_df[anno_df['unchar_phylum']==True]['phylum'].nunique()

216

In [127]:
anno_df[anno_df['unchar_phylum']==True]['superkingdom'].value_counts(normalize=True)

superkingdom
Bacteria     0.596241
Archaea      0.207197
Eukaryota    0.195324
Viruses      0.001238
Name: proportion, dtype: float64

In [128]:
anno_df['unchar_class'] = anno_df['lineage'].apply(lambda x: uncharacterized_tax_group(x, 'class'))

C:\Users\terca\AppData\Local\Temp\ipykernel_856\289151479.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anno_df['unchar_class'] = anno_df['lineage'].apply(lambda x: uncharacterized_tax_group(x, 'class'))


In [129]:
anno_df['unchar_order'] = anno_df['lineage'].apply(lambda x: uncharacterized_tax_group(x, 'order'))

C:\Users\terca\AppData\Local\Temp\ipykernel_856\3386436911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anno_df['unchar_order'] = anno_df['lineage'].apply(lambda x: uncharacterized_tax_group(x, 'order'))


In [130]:
anno_df['unchar_family'] = anno_df['lineage'].apply(lambda x: uncharacterized_tax_group(x, 'family'))

C:\Users\terca\AppData\Local\Temp\ipykernel_856\716035138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anno_df['unchar_family'] = anno_df['lineage'].apply(lambda x: uncharacterized_tax_group(x, 'family'))


In [131]:
anno_df['unchar_genus'] = anno_df['lineage'].apply(lambda x: uncharacterized_tax_group(x, 'genus'))

C:\Users\terca\AppData\Local\Temp\ipykernel_856\1155281583.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anno_df['unchar_genus'] = anno_df['lineage'].apply(lambda x: uncharacterized_tax_group(x, 'genus'))


In [136]:
anno_df['unchar_species'] = anno_df['lineage'].apply(lambda x: uncharacterized_tax_group(x, 'species'))

C:\Users\terca\AppData\Local\Temp\ipykernel_856\222861725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anno_df['unchar_species'] = anno_df['lineage'].apply(lambda x: uncharacterized_tax_group(x, 'species'))


In [137]:
anno_df

,id,organism,tax_id,lineage,superkingdom,kingdom,phylum,unchar_phylum,unchar_class,unchar_order,unchar_family,unchar_genus,unchar_species
0,bfd_106193,soil metagenome,410658,{'species': 'soil metagenome'},Metagenome,environmental,unknown,None,None,None,None,None,True
1,bfd_118229,soil metagenome,410658,{'species': 'soil metagenome'},Metagenome,environmental,unknown,None,None,None,None,None,True
2,bfd_059047,unknown,error,error,unknown,unknown,unknown,None,None,None,None,None,None
3,bfd_096880,Absidia repens,90262,"{'species': 'Absidia repens', 'genus': 'Absidi...",Eukaryota,Fungi,Mucoromycota,True,True,True,True,True,True
4,bfd_097326,soil metagenome,410658,{'species': 'soil metagenome'},Metagenome,environmental,unknown,None,None,None,None,None,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
682466,uniprot_163936,Corynebacterium aquilae DSM 44791,1431546,"{'species': 'Corynebacterium aquilae', 'genus'...",Bacteria,unknown,Actinomycetota,False,False,False,True,True,True
682467,uniprot_188186,Pygocentrus nattereri,42514,"{'species': 'Pygocentrus nattereri', 'genus': ...",Eukaryota,Metazoa,Chordata,False,True,True,True,True,True
682468,uniprot_229305,Treponema sp.,166,"{'species': 'Treponema sp.', 'genus': 'Trepone...",Bacteria,unknown,Spirochaetota,True,True,True,True,True,True
682469,uniprot_181461,Photobacterium phosphoreum,659,"{'species': 'Photobacterium phosphoreum', 'gen...",Bacteria,unknown,Pseudomonadota,False,False,True,True,True,True


In [138]:
def get_tax_score(x):
    if x['unchar_phylum'] == True:
        return 1
    elif x['unchar_class'] == True:
        return 0.8
    elif x['unchar_order'] == True:
        return 0.6
    elif x['unchar_family'] == True:
        return 0.4
    elif x['unchar_genus'] == True:
        return 0.2
    elif x['unchar_species'] == False:
        return 0
    elif x['superkingdom'] == 'Metagenome':
        return 0
    elif x['superkingdom'] == 'unknown':
        return 0
    elif x['unchar_species'] == True:
        return 0.1
    else:
        return 0

In [140]:
anno_df['tax_score'] = anno_df.apply(lambda x: get_tax_score(x), axis=1)

C:\Users\terca\AppData\Local\Temp\ipykernel_856\790180379.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anno_df['tax_score'] = anno_df.apply(lambda x: get_tax_score(x), axis=1)


In [147]:
anno_df[anno_df['superkingdom'] == 'Eukaryota'].sort_values(by='tax_score', ascending=False)

,id,organism,tax_id,lineage,superkingdom,kingdom,phylum,unchar_phylum,unchar_class,unchar_order,unchar_family,unchar_genus,unchar_species,tax_score
3,bfd_096880,Absidia repens,90262,"{'species': 'Absidia repens', 'genus': 'Absidi...",Eukaryota,Fungi,Mucoromycota,True,True,True,True,True,True,1.0
457422,tsa_041694,Hirudo medicinalis,6421,"{'species': 'Hirudo medicinalis', 'genus': 'Hi...",Eukaryota,Metazoa,Annelida,True,True,True,True,True,True,1.0
457479,tsa_040094,Evechinus chloroticus,137513,"{'species': 'Evechinus chloroticus', 'genus': ...",Eukaryota,Metazoa,Echinodermata,True,True,True,True,True,True,1.0
457477,tsa_057529,Stauroneis constricta,265584,"{'species': 'Stauroneis constricta', 'genus': ...",Eukaryota,unknown,Bacillariophyta,True,True,True,True,True,True,1.0
457475,tsa_048409,Spirobranchus lamarcki,2082999,"{'species': 'Spirobranchus lamarcki', 'genus':...",Eukaryota,Metazoa,Annelida,True,True,True,True,True,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484035,uniprot_016265,Nicotiana sylvestris,4096,"{'species': 'Nicotiana sylvestris', 'genus': '...",Eukaryota,Viridiplantae,Streptophyta,False,False,False,False,False,False,0.0
484043,uniprot_041927,Olea europaea subsp. europaea,158383,"{'species': 'Olea europaea', 'genus': 'Olea', ...",Eukaryota,Viridiplantae,Streptophyta,False,False,False,False,False,False,0.0
42887,bfd_029749,Metarhizium majus (strain ARSEF 297),1650735,"{'species': 'Metarhizium majus', 'genus': 'Met...",Eukaryota,Fungi,Ascomycota,False,False,False,False,False,False,0.0
484055,uniprot_017026,Gossypium arboreum,29729,"{'species': 'Gossypium arboreum', 'genus': 'Go...",Eukaryota,Viridiplantae,Streptophyta,False,False,False,False,False,False,0.0


In [150]:
anno_df[(anno_df['superkingdom'] == 'Eukaryota') & (anno_df['unchar_phylum'] == True)]['phylum'].value_counts()

phylum
Bacillariophyta       1099
Nematoda               852
Mollusca               724
Haptophyta             708
Mucoromycota           684
Platyhelminthes        382
Rotifera               323
Oomycota               284
Euglenozoa             242
Apicomplexa            242
Echinodermata          211
Annelida               177
Chytridiomycota        134
Ciliophora             110
Zoopagomycota          108
Cercozoa                92
Foraminifera            83
Discosea                72
Heterolobosea           59
Blastocladiomycota      43
Tardigrada              39
Ctenophora              37
Bryozoa                 24
Hemichordata            22
Onychophora             20
Xenacoelomorpha         18
Prasinodermophyta       17
Entoprocta              17
Placozoa                15
Microsporidia           13
Perkinsozoa             11
Brachiopoda             10
Parabasalia             10
Endomyxa                10
Phoronida                9
Fornicata                8
Priapulida           

In [143]:
anno_df.sort_values(by='tax_score', ascending=False)

,id,organism,tax_id,lineage,superkingdom,kingdom,phylum,unchar_phylum,unchar_class,unchar_order,unchar_family,unchar_genus,unchar_species,tax_score
578174,uniprot_140309,Desulfurobacterium sp. TC5-1,1158318,"{'species': 'Desulfurobacterium sp. TC5-1', 'g...",Bacteria,unknown,Aquificota,True,True,True,True,True,True,1.0
31998,bfd_007245,Euryarchaeota archaeon,2026739,"{'species': 'Euryarchaeota archaeon', 'phylum'...",Archaea,unknown,Euryarchaeota,True,None,None,None,None,True,1.0
139767,bfd_196253,Micavibrio sp,2024841.0,"{'species': 'Micavibrio sp.', 'genus': 'Micavi...",Bacteria,unknown,Bdellovibrionota,True,True,True,True,True,True,1.0
510819,uniprot_053734,Candidatus Poseidoniales archaeon,2163009,{'species': 'Candidatus Poseidoniales archaeon...,Archaea,unknown,Candidatus Thermoplasmatota,True,True,True,None,None,True,1.0
623412,uniprot_189885,Halobellus sp. Atlit-31R,2282130,"{'species': 'Halobellus sp. Atlit-31R', 'genus...",Archaea,unknown,Euryarchaeota,True,True,True,True,True,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254447,mgnify_new_182426,unknown,unknown,root:Environmental:Aquatic:Freshwater:Pond:Sed...,Metagenome,Environmental,unknown,None,None,None,None,None,None,0.0
254448,mgnify_new_143774,unknown,unknown,root:Mixed,Metagenome,Mixed,unknown,None,None,None,None,None,None,0.0
254449,mgnify_new_155551,unknown,unknown,root:Mixed,Metagenome,Mixed,unknown,None,None,None,None,None,None,0.0
254450,mgnify_new_112974,unknown,unknown,NaN,Metagenome,NaN,unknown,None,None,None,None,None,None,0.0


In [153]:
anno_df.to_csv('all_annotation_with_tax_score.csv')

In [9]:
anno_df[anno_df['kingdom']=='Metazoa']['phylum'].value_counts()

phylum
Arthropoda         14102
Chordata            7616
Nematoda             852
Mollusca             724
Cnidaria             551
Platyhelminthes      382
Rotifera             323
Echinodermata        211
Annelida             177
Porifera              89
Tardigrada            39
Ctenophora            37
Bryozoa               24
Hemichordata          22
Onychophora           20
Xenacoelomorpha       18
Entoprocta            17
Placozoa              15
Brachiopoda           10
Phoronida              9
Priapulida             6
Dicyemida              5
Nemertea               3
Acanthocephala         1
Orthonectida           1
Name: count, dtype: int64

In [17]:
anno_df[anno_df['phylum']=='Chordata'].iloc[0]['lineage']

{'species': 'Eptesicus fuscus',
 'genus': 'Eptesicus',
 'family': 'Vespertilionidae',
 'order': 'Chiroptera',
 'class': 'Mammalia',
 'phylum': 'Chordata',
 'kingdom': 'Metazoa',
 'superkingdom': 'Eukaryota'}

In [37]:
anno_df['class'] = anno_df['lineage'].apply(lambda x:  x.get('class', None) if type(x) == dict else None)

In [38]:
anno_df['order'] = anno_df['lineage'].apply(lambda x:  x.get('order', None) if type(x) == dict else None)

In [29]:
anno_df[anno_df['class'] == 'Mammalia']['order'].value_counts().sort_values(ascending=False).head(50)

order
Rodentia           579
Artiodactyla       490
Primates           426
Carnivora          383
Chiroptera         193
Lagomorpha          69
Perissodactyla      52
Eulipotyphla        37
Diprotodontia       25
Dasyuromorphia      21
Proboscidea         20
Monotremata         13
Pilosa              12
Didelphimorphia     12
Scandentia          10
Pholidota           10
Microbiotheria       8
Dermoptera           6
Tubulidentata        6
Cingulata            4
Sirenia              4
Macroscelidea        3
Hyracoidea           1
Peramelemorphia      1
Name: count, dtype: int64

In [32]:
anno_df[anno_df['phylum']=='Chordata']['class'].value_counts()

class
Actinopteri       2947
Mammalia          2394
Aves              1050
Amphibia           432
Lepidosauria       330
Chondrichthyes     107
Leptocardii         32
Hyperoartia         26
Ascidiacea          19
Cladistia           17
Appendicularia       8
Myxini               4
Thaliacea            3
Name: count, dtype: int64

In [40]:
anno_df.columns

Index(['id', 'architecture_pfam', 'architecture_l_pfam', 'PF06330.14',
       'PF01397.24', 'PF03936.19', 'PF00494.22', 'PF13249.9', 'PF19086.3',
       'PF13243.9', 'architecture_supfam', 'architecture_l_supfam', '0041184',
       '0053354', '0053355', '0048261', '0048806', '0046340', '0047573',
       'record_id', 'record_description', 'new_id', 'organism', 'tax_id',
       'lineage', 'superkingdom', 'kingdom', 'phylum', 'class', 'order'],
      dtype='object')

In [42]:
pfam_cols = ['PF06330.14','PF01397.24', 'PF03936.19', 'PF00494.22', 'PF13249.9', 'PF19086.3','PF13243.9']

The hits are probably oxidosqualene synthases or squalene synthases (PF00494.22, PF13249.9, PF13243.9).

Some entries from UniParc are annotated with Rat but mostly appear in Arabidopsis.

In [47]:
# Fish
anno_df[anno_df['class'] == 'Actinopteri'][pfam_cols].fillna(0).apply(lambda x: x.value_counts(normalize=True)).fillna(0).loc[[1.0]]

,PF06330.14,PF01397.24,PF03936.19,PF00494.22,PF13249.9,PF19086.3,PF13243.9
1.0,0.0,0.001357,0.017645,0.013573,0.281303,0.002715,0.281303


In [48]:
# Mammals
anno_df[anno_df['class'] == 'Mammalia'][pfam_cols].fillna(0).apply(lambda x: x.value_counts(normalize=True)).fillna(0).loc[[1.0]]

,PF06330.14,PF01397.24,PF03936.19,PF00494.22,PF13249.9,PF19086.3,PF13243.9
1.0,0.000418,0.022556,0.027987,0.011278,0.307435,0.02381,0.307018


In [49]:
# Birds
anno_df[anno_df['class'] == 'Aves'][pfam_cols].fillna(0).apply(lambda x: x.value_counts(normalize=True)).fillna(0).loc[[1.0]]

,PF06330.14,PF01397.24,PF03936.19,PF00494.22,PF13249.9,PF19086.3,PF13243.9
1.0,0.0,0.0,0.0,0.004762,0.308571,0.0,0.311429


In [51]:
mammals = anno_df[anno_df['class'] == 'Mammalia']

In [54]:
mammals[mammals['PF06330.14']==1]

,id,architecture_pfam,architecture_l_pfam,PF06330.14,PF01397.24,PF03936.19,PF00494.22,PF13249.9,PF19086.3,PF13243.9,...,record_description,new_id,organism,tax_id,lineage,superkingdom,kingdom,phylum,class,order
49706,bfd_004444,['PF19086.3'],['PF19086.3'],1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,UPI00033410EE status=active,bfd_004444,Octodon degus (Degu),10160,"{'species': 'Octodon degus', 'genus': 'Octodon...",Eukaryota,Metazoa,Chordata,Mammalia,Rodentia


In [59]:
mammals.columns

Index(['id', 'architecture_pfam', 'architecture_l_pfam', 'PF06330.14',
       'PF01397.24', 'PF03936.19', 'PF00494.22', 'PF13249.9', 'PF19086.3',
       'PF13243.9', 'architecture_supfam', 'architecture_l_supfam', '0041184',
       '0053354', '0053355', '0048261', '0048806', '0046340', '0047573',
       'record_id', 'record_description', 'new_id', 'organism', 'tax_id',
       'lineage', 'superkingdom', 'kingdom', 'phylum', 'class', 'order'],
      dtype='object')

In [61]:
mammals[mammals['PF01397.24']==1]['organism'].value_counts()

organism
Rattus norvegicus    54
Name: count, dtype: int64

In [62]:
mammals[mammals['PF03936.19']==1]['organism'].value_counts()

organism
Rattus norvegicus                                                             60
Chinchilla lanigera                                                            3
Octodon degus (Degu)                                                           1
Mus musculus musculus (eastern European house mouse); Mus musculus (Mouse)     1
Cavia porcellus (Guinea pig)                                                   1
Octodon degus                                                                  1
Name: count, dtype: int64

In [66]:
mammals[mammals['PF19086.3']==1][['record_id','organism']]

,record_id,organism
32431,UPI0003F0F688,Chrysochloris asiatica (Cape golden mole)
49706,UPI00033410EE,Octodon degus (Degu)
463626,Q9LVP7;,Rattus norvegicus
464841,Q9LH59,Rattus norvegicus
466190,A0A1P8B0J1,Rattus norvegicus
467254,O22680.1,Rattus norvegicus
467707,O65434,Rattus norvegicus
469171,NoUniProtID,Rattus norvegicus
470545,Q9LS76.1,Rattus norvegicus
473875,Q84UU4-2;,Rattus norvegicus
